### Aggregation 함수 및 GroupBy 적용

#### Aggregation 함수

In [2]:
import numpy as np
import pandas as pd

# 여러 행 동시에 입력받기 위해 가져옴
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

titanic_df = pd.read_csv("titanic_train.csv")

In [3]:
# NaN값은 count에서 제외
titanic_df.count()

PassengerId    891
Survived       891
Pclass         891
Name           891
Sex            891
Age            714
SibSp          891
Parch          891
Ticket         891
Fare           891
Cabin          204
Embarked       889
dtype: int64

특정 컬럼들로 Aggregation 함수 수행.

In [4]:
titanic_df[["Age", "Fare"]].mean(axis = 0)
titanic_df[["Age", "Fare"]].sum(axis = 0)
titanic_df[['Age', 'Fare']].count()

Age     29.699118
Fare    32.204208
dtype: float64

Age     21205.1700
Fare    28693.9493
dtype: float64

Age     714
Fare    891
dtype: int64

#### groupby( )  
by 인자에 Group By 하고자 하는 컬럼을 입력, 여러개의 컬럼으로 Group by 하고자 하면 [ ] 내에 해당 컬럼명을 입력. DataFrame에 groupby( )를 호출하면 DataFrameGroupBy 객체를 반환.

In [5]:
titanic_groupby = titanic_df.groupby(by = "Pclass")
print(type(titanic_groupby)) # DataFrameGroupBy 객체
print(titanic_groupby)

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>


DataFrameGroupBy객체에 Aggregation함수를 호출하여 Group by 수행.

In [6]:
titanic_groupby2 = titanic_df.groupby(by = "Pclass").count()
titanic_groupby2

,PassengerId,Survived,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
Pclass,,,,,,,,,,,
1,216,216,216,216,186,216,216,216,216,176,214
2,184,184,184,184,173,184,184,184,184,16,184
3,491,491,491,491,355,491,491,491,491,12,491


In [7]:
print(type(titanic_groupby2))
print(titanic_groupby2.shape)
print(titanic_groupby2.index)

<class 'pandas.core.frame.DataFrame'>
(3, 11)
Int64Index([1, 2, 3], dtype='int64', name='Pclass')


In [8]:
titanic_groupby3 = titanic_df.groupby(by = "Pclass")[['PassengerId', 'Survived']].count()
titanic_groupby3

,PassengerId,Survived
Pclass,,
1,216,216
2,184,184
3,491,491


In [9]:
titanic_df[['Pclass','PassengerId', 'Survived']].groupby('Pclass').count()

,PassengerId,Survived
Pclass,,
1,216,216
2,184,184
3,491,491


In [10]:
titanic_df.groupby('Pclass')['Pclass'].count()
titanic_df['Pclass'].value_counts() # value_counts()는 Series에만 적용 가능

Pclass
1    216
2    184
3    491
Name: Pclass, dtype: int64

3    491
1    216
2    184
Name: Pclass, dtype: int64

RDBMS의 group by는 select 절에 여러개의 aggregation 함수를 적용할 수 있음.

Select max(Age), min(Age) from titanic_table group by Pclass

판다스는 여러개의 aggregation 함수를 적용할 수 있도록 agg( )함수를 별도로 제공

In [11]:
titanic_df.groupby("Pclass")["Age"].agg([max, min])

,max,min
Pclass,,
1,80.0,0.92
2,70.0,0.67
3,74.0,0.42


딕셔너리를 이용하여 다양한 aggregation 함수를 적용

In [12]:
agg_format = {"Age" : "max", "SibSp" : "sum", "Fare" : "mean"}
titanic_df.groupby("Pclass").agg(agg_format)

,Age,SibSp,Fare
Pclass,,,
1,80.0,90,84.154687
2,70.0,74,20.662183
3,74.0,302,13.675550


### 결손(Missing) 데이터 처리하기
DataFrame의 isna( ) 메소드는 모든 컬럼값들이 NaN인지 True/False값을 반환한다(NaN이면 True).

In [13]:
titanic_df.isna().head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,False,False,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,True,False


아래와 같이 isna( ) 반환 결과에 sum( )을 호출하여 컬럼별로 NaN 건수를 구할 수 있다.

In [14]:
titanic_df.isna( ).sum( )

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

### fillna( ) 로 Missing 데이터 대체하기

In [15]:
titanic_df["Cabin"] = titanic_df["Cabin"].fillna("C000", inplace = False)
titanic_df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,C000,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,C000,S


In [16]:
titanic_df["Age"] = titanic_df["Age"].fillna(titanic_df["Age"].mean())
titanic_df['Embarked'] = titanic_df['Embarked'].fillna('S')
titanic_df.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

## apply lambda 식으로 데이터 가공
파이썬 lambda 식 기본

In [17]:
lambda_square = lambda x : x ** 2
print("3의 제곱은 :", lambda_square(3))

3의 제곱은 : 9


In [18]:
a = [1,2,3]
squares = map(lambda x : x**2, a)
list(squares)

[1, 4, 9]

#### 판다스에 apply lambda 식 적용

In [19]:
titanic_df["Name_len"] = titanic_df["Name"].apply(lambda x : len(x))
titanic_df["Name_len"].head(3)

0    23
1    51
2    22
Name: Name_len, dtype: int64

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Name_len
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,C000,S,23
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C85,C,51
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,C000,S,22
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,C123,S,44
4,5,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,C000,S,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.000000,0,0,211536,13.0000,C000,S,21
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.000000,0,0,112053,30.0000,B42,S,28
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,29.699118,1,2,W./C. 6607,23.4500,C000,S,40
889,890,1,1,"Behr, Mr. Karl Howell",male,26.000000,0,0,111369,30.0000,C148,C,21
